In [ ]:
import argparse
import collections
import copy
import numpy as np
import sklearn.metrics as metrics
import tensorboardX as tb
import torch as th
import torch.nn as nn
import torch.utils as utils
import data
import lenet
import my
import resnet

In [ ]:
args = argparse.Namespace()
args.actor = 'linear'
args.avg = 'binary' # average
args.bsa = 100 # batch size of actor
args.ckpt_id = "parameter" \
"#actor:linear#avg:binary#bsa:100#bscx:1#bscy:1#ds:cifar10#iw:none#lbl:91#lra:0.001#lrc:0.001#ni:10000#nia:1#nic:25#np:25#ssc:1#std:7.5#tau:0.1-actor-1000"
args.ds = 'cifar10' # data set
args.gpu = 0
args.lbl = '91' # labelling
args.np = 25 # number of perturbations
# args.p = 'a' # perturbation: actor
args.p = 'p' # perturbation: parameter
args.std = 0.1
args.tensorboard = True

keys = sorted(vars(args).keys())
excluded = ['ckpt_id', 'gpu', 'tensorboard']
experiment_id = 'perturbation#' + '#'.join('%s:%s' % (key, str(getattr(args, key))) for key in keys if key not in excluded)
if args.tensorboard:
    writer = tb.SummaryWriter('runs/' + experiment_id)

In [ ]:
if args.gpu < 0:
    cuda = False
    new_tensor = th.FloatTensor
else:
    cuda = True
    new_tensor = th.cuda.FloatTensor
    th.cuda.set_device(args.gpu)

labelling = {} if args.lbl == '' else {(0, 9) : 0, (9, 10) : 1}
rbg = args.actor in ('lenet', 'resnet')
train_x, train_y, test_x, test_y = getattr(data, 'load_%s' % args.ds)(labelling, rbg, torch=True)

train_set = utils.data.TensorDataset(train_x, train_y)
train_loader = utils.data.DataLoader(train_set, 4096, drop_last=False)
test_set = utils.data.TensorDataset(test_x, test_y)
test_loader = utils.data.DataLoader(test_set, 4096, drop_last=False)

loader = data.BalancedDataLoader(train_x, train_y, args.bsa, cuda)

n_classes = int(train_y.max() - train_y.min() + 1)

In [ ]:
def global_scores(c, loader):
    key_list = ['accuracy', 'precision', 'recall', 'f1']
    score_list = [
        metrics.accuracy_score,
        lambda y, y_bar: metrics.precision_recall_fscore_support(y, y_bar, average=args.avg)
    ]
    accuracy, (precision, recall, f1, _) = my.global_scores(c, loader, score_list)
    return collections.OrderedDict({
        'accuracy'  : accuracy,
        'precision' : precision,
        'recall'    : recall,
        'f1'        : f1,
    })

def report(actor, i):
    train_scores = global_scores(actor, train_loader)
    test_scores = global_scores(actor, test_loader)

    prefix = '0' * (len(str(args.np)) - len(str(i + 1)))
    print('[iteration %s%d]' % (prefix, i + 1) + \
          ' | '.join('%s %0.3f/%0.3f' % (key, value, test_scores[key]) for key, value in train_scores.items()))

    if args.tensorboard:
        for key, value in train_scores.items():
            writer.add_scalar('train-' + key, value, i + 1)
        for key, value in test_scores.items():
            writer.add_scalar('test-' + key, value, i + 1)

In [ ]:
th.random.manual_seed(1)
if cuda:
    th.cuda.manual_seed_all(1)

n_channels = 1 if args.ds == 'mnist' else 3
size = 28 if args.ds == 'mnist' else 32
actor = {
    'linear' : nn.Linear(n_channels * size ** 2, n_classes),
    'lenet'  : lenet.LeNet(3, n_classes, size),
    'resnet' : resnet.ResNet(depth=18, n_classes=n_classes),
}[args.actor]
if args.ckpt_id:
    actor.load_state_dict(th.load('ckpt/' + args.ckpt_id))
my.set_requires_grad(actor, False)
if cuda:
    actor.cuda()

report(actor, -1)

In [ ]:
for i in range(args.np):
    x, y = next(loader)
    if args.p == 'p':
        my.perturb(actor, args.std)
    z = actor(x)
    if args.p == 'a':
        z = z + args.std * th.randn(z.shape, device=z.device)
    y_bar = th.max(z, 1)[1]
    
#     statistics = metrics.precision_recall_fscore_support(y, y_bar)
#     statistics = map(np.ndarray.tolist, statistics)
#     statistics = zip(*statistics)
#     for j, s in enumerate(statistics):
#         precision, recall, f1_score, support = s
#         writer.add_scalar('precision-%d' % j, precision, i + 1)
#         writer.add_scalar('recall-%d' % j, recall, i + 1)
#         writer.add_scalar('f1_score-%d' % j, f1_score, i + 1)
#         writer.add_scalar('support-%d' % j, support, i + 1)

In [ ]:
metrics.confusion_matrix(y, y_bar)